In [4]:
import numpy as np
import random
import timeit
import math
from scipy import stats

### Criação da lista de variáveis aleatórias

In [2]:
# Gerará uma lista de variáveis aleatórias a partir da função RANDOM
def G5(n):
    random.seed(0)

    vet_rn = []
    for i in range(0,n,1):
        vet_rn.append(random.random())
    
    return(vet_rn)

In [3]:
# Transforma uma lista de variáveis aleatórias em uma distribuição aleatória uniforme entre [0,1]
def random_list(vet,B):
    Ts = np.zeros(len(vet))
    for i in range(0,len(vet)):
        Ts[i] = -B*np.log(1-vet[i])
    return list(Ts)

### Função para formação do intervalo de entrada

In [6]:
# Soma-se o próximo com o anterior para ter uma noção de tempo de chagada. Sempre o tempo de chegada do
#primeiro será 0.
def sum_intervals(vet):
    for i in range(1, len(vet)):
        vet[i] += vet[i-1]
    return vet

### Função para calcular intervalo de confiança

In [9]:
# Dado uma lista de valores de processamento MM1, calcula-se com a função stats.t.interval o intervalo
#de confiança para 95%
def calculate_confidence_95(mm1):
    return (stats.t.interval(confidence=0.95, df = len(mm1)-1, loc=np.mean(mm1), scale=stats.sem(mm1)))

### Fila MM1

In [36]:
def process_queue(clock, wait, tx_service, tx_entry):
    # Wait = Tempo de espera total
    # tx_service = Lista com taxas de serviço
    # tx_entry = Lista com taxas de entrada
    # qtd_iteractions = Total de elementos processados
    # wait_list = histórico de tempos de espera
    # is_run = Flag que serve para quebrar o loop infinito de chamada dessa função
    for i in range(0,len(tx_service)):
        wait += clock - tx_entry[i] #Tempo de espera atual = Clcok - tx_entrada atual
        clock += tx_service[i] #Adiciona-se ao clock a taxa de serviço atual
        print("Wait:",wait)
        print("Clock: ",clock,"\n")
        
    return wait

In [37]:
# Teste da fila MM1
clock = 0
wait = 0
tx_service = [5,5,5,5]
tx_entry = [0,3,7,8]
process_queue(clock, wait, tx_service, tx_entry)

Wait: 0
Clock:  5 

Wait: 2
Clock:  10 

Wait: 5
Clock:  15 

Wait: 12
Clock:  20 



12